<a href="https://colab.research.google.com/github/charu210703/Rough-Decision-Making-of-Facial-Expression-Detection/blob/main/Rough_Decision_Making_of_Facial_Expression_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As problems evolve in complexity, there’s a growing need for robust
methods to manage uncertain, incomplete, and imprecise information.
The motivation behind our proposed model lies in the quest for a
nuanced approach to handling decision making problems, especially in
the presence of uncertain information.


Special thanks are extended to Dr. Shan Li and Dr. Weihong Dong
for providing the RAF-DB database, enabling the implementation of
our mathematical model. As the dataset is private and only available for non-commercial research purposes only, we wouldn't be able to upload it in a public repo. <br> Let the dataset.csv be a csv file we have generated from the dataset, containing the following parameters:<br>


1.   Image name
2.   Age
3.   Gender
4.   Emotion (7 classes of basic emotions)
5.   x and y coordinates of 5 accurate landmark locations
6.   x and y coordinates of 37 automatic landmark locations


In [ ]:
import pandas as pd
import numpy as np
import math
df = pd.read_csv('<dataset csv file>')

We focused on five key facial parameters using geometric features and utilized coordinates of facial landmarks to calculate parameters.
The parameters include areas between the eye and eyebrow, eyelids, lip
landmarks, and distances between facial features.

In [ ]:
def calc_area_eye_eyebrow_points(row):
  x_coordinates_l = row[['eb_l_r_x','eb_l_c_x','eb_l_l_x','eye_l_r_most_x','eye_l_up_r_x','eye_l_up_l_x','eye_l_l_most_x']]
  y_coordinates_l = row[['eb_l_r_y','eb_l_c_y','eb_l_l_y','eye_l_r_most_y','eye_l_up_r_y','eye_l_up_l_y','eye_l_l_most_y']]
  x_coordinates_l = np.append(x_coordinates_l, x_coordinates_l[0])  # Close the polygon
  y_coordinates_l = np.append(y_coordinates_l, y_coordinates_l[0])
  area_l = 0.5 * np.abs(np.sum(x_coordinates_l[:-1] * y_coordinates_l[1:] - x_coordinates_l[1:] * y_coordinates_l[:-1]))
  x_coordinates_r = row[['eb_r_r_x','eb_r_c_x','eb_r_l_x','eye_r_r_most_x','eye_r_up_r_x','eye_r_up_l_x','eye_r_l_most_x']]
  y_coordinates_r = row[['eb_r_r_y','eb_r_c_y','eb_r_l_y','eye_r_r_most_y','eye_r_up_r_y','eye_r_up_l_y','eye_r_l_most_y']]
  x_coordinates_r = np.append(x_coordinates_r, x_coordinates_r[0])  # Close the polygon
  y_coordinates_r = np.append(y_coordinates_r, y_coordinates_r[0])
  area_r = 0.5 * np.abs(np.sum(x_coordinates_r[:-1] * y_coordinates_r[1:] - x_coordinates_r[1:] * y_coordinates_r[:-1]))
  return np.mean([area_l, area_r])

df['area_eye_eyebrow_points'] = df.apply(calc_area_eye_eyebrow_points, axis=1)

In [ ]:
def calc_area_eye_points(row):
  x_coordinates_l = row[['eye_l_dow_r_x','eye_l_dow_l_x','eye_l_r_most_x','eye_l_up_r_x','eye_l_up_l_x','eye_l_l_most_x']]
  y_coordinates_l = row[['eye_l_dow_r_y','eye_l_dow_l_y','eye_l_r_most_y','eye_l_up_r_y','eye_l_up_l_y','eye_l_l_most_y']]
  x_coordinates_l = np.append(x_coordinates_l, x_coordinates_l[0])  # Close the polygon
  y_coordinates_l = np.append(y_coordinates_l, y_coordinates_l[0])
  area_l = 0.5 * np.abs(np.sum(x_coordinates_l[:-1] * y_coordinates_l[1:] - x_coordinates_l[1:] * y_coordinates_l[:-1]))
  x_coordinates_r = row[['eye_r_dow_r_x','eye_r_dow_l_x','eye_r_r_most_x','eye_r_up_r_x','eye_r_up_l_x','eye_r_l_most_x']]
  y_coordinates_r = row[['eye_r_dow_r_y','eye_r_dow_l_y','eye_r_r_most_y','eye_r_up_r_y','eye_r_up_l_y','eye_r_l_most_y']]
  x_coordinates_r = np.append(x_coordinates_r, x_coordinates_r[0])  # Close the polygon
  y_coordinates_r = np.append(y_coordinates_r, y_coordinates_r[0])
  area_r = 0.5 * np.abs(np.sum(x_coordinates_r[:-1] * y_coordinates_r[1:] - x_coordinates_r[1:] * y_coordinates_r[:-1]))
  return np.mean([area_l, area_r])

df['area_eye_points'] = df.apply(calc_area_eye_points, axis=1)

In [ ]:
def infer_lip_landmarks(row):
    upper_lip_points = np.array(row[['lip_up_1_x', 'lip_up_1_y', 'lip_up_2_x', 'lip_up_2_y', 'lip_up_3_x', 'lip_up_3_y']])
    lower_lip_points = np.array(row[['lip_dow_1_x', 'lip_dow_1_y', 'lip_dow_2_x', 'lip_dow_2_y', 'lip_dow_3_x', 'lip_dow_3_y']])

    upper_midpoint = np.mean(upper_lip_points.reshape(3, 2), axis=0)
    lower_midpoint = np.mean(lower_lip_points.reshape(3, 2), axis=0)
    vertical_distance = np.linalg.norm(upper_midpoint - lower_midpoint)
    return vertical_distance

df['lip_landmarks_distance'] = df.apply(infer_lip_landmarks, axis=1)

In [ ]:
#two types of nose wrinkle

df['distance_eb_cen_nose_cen'] = np.sqrt(
    (df['eb_cen_x'] - df['nose_cen_x'])**2 + (df['eb_cen_y'] - df['nose_cen_y'])**2
)

df['distance_eb_cen_nose_tip'] = np.sqrt(
    (df['eb_cen_x'] - df['nose_tip_x'])**2 + (df['eb_cen_y'] - df['nose_tip_y'])**2
)

There 12271 images for training and 3068 images for testing.

In [ ]:
test_data = df[df['img_name'].str.startswith('test')]
train_data = df[df['img_name'].str.startswith('train')]

As this is an affective dataset and some of the images are turned away or unclear, we are only taking the images which has not much deviation from the 3 centre points of the face.

In [ ]:
df['vals'] = df[['eb_cen_x','nose_cen_x','lip_up_3_x','lip_dow_3_x']].values.max(1) - df[['eb_cen_x','nose_cen_x','lip_up_3_x','lip_dow_3_x']].values.min(1)
df = df.loc[df['vals'] <= 1]

We are splitting the data into the 7 labels given.<br>
Emotion labels:<br>
1: Surprise<br>
2: Fear<br>
3: Disgust<br>
4: Happiness<br>
5: Sadness<br>
6: Anger<br>
7: Neutral<br>
Confidence intervals are derived to quantify variability in each emotion
across facial features.


In [ ]:
sur = train_data[train_data['emotion'] == 1]
fear = train_data[train_data['emotion'] == 2]
disg = train_data[train_data['emotion'] == 3]
happ = train_data[train_data['emotion'] == 4]
sad = train_data[train_data['emotion'] == 5]
ang = train_data[train_data['emotion'] == 6]
neu = train_data[train_data['emotion'] == 7]

In [ ]:
params = ['area_eye_eyebrow_points','area_eye_points','lip_landmarks_distance','distance_eb_cen_nose_cen','distance_eb_cen_nose_tip']
emos = ['sur','fear','disg','happ','sad','ang','neu']

def calc_confi_int(params, emo_data):
  res = []
  for p in params:
    data = emo_data[p]
    m = np.mean(data)
    se = np.std(data) / math.sqrt(len(data))
    ci = [m - se * 3.891, m + se * 3.891]
    res.append(ci)
  return res


In [ ]:
sur_ci = calc_confi_int(params,sur)
fear_ci = calc_confi_int(params,fear)
disg_ci = calc_confi_int(params,disg)
happ_ci = calc_confi_int(params,happ)
sad_ci = calc_confi_int(params,sad)
ang_ci = calc_confi_int(params,ang)
neu_ci = calc_confi_int(params,neu)

emo_ci = []
emo_ci.append(sur_ci)
emo_ci.append(fear_ci)
emo_ci.append(disg_ci)
emo_ci.append(happ_ci)
emo_ci.append(sad_ci)
emo_ci.append(ang_ci)
emo_ci.append(neu_ci)

Then we are calculating the upper and lower approximation spaces.

In [ ]:
epsilon = 1

def calc_approx_spaces(data):
    low = []
    up = set()
    d = []

    for i in range(7):
        emo = emo_ci[i]
        emo_ct = 0

        for j in range(5):
            interv = [data[j + 1] + epsilon, data[j + 1] - epsilon]

            if emo[j][0] <= interv[0] and interv[1] <= emo[j][1]:
                emo_ct += 1

        # Decide whether to add the emotion to "low" or "up" based on emo_ct
        if emo_ct >= 3:
            low.append(emos[i])
        elif emo_ct > 0 and emo_ct < 3:
            up.add(emos[i])

        d.append([emos[i], emo_ct])

    return (low, up, d)

test_data_params[['lower_approx', 'upper_approx', 'emo_data']] = test_data_params.apply(calc_approx_spaces, axis=1, result_type='expand')


Thus this approach gives us the set of possible emotion that particular image can have.